In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
response= requests.get('https://www.imdb.com/search/title/?title_type=feature&num_votes=5000,&languages=en&sort=boxoffice_gross_us,desc&explore=genres&view=advanced')
soup = BeautifulSoup(response.content, "html.parser")

# Find the table element you want to scrape
movies_divs = soup.find_all("div", class_="lister-item-content")

movies=[]
for movie in movies_divs:
    title=movie.find('a').text
    year=movie.find('span', class_='lister-item-year').text.strip('()')
    runtime=movie.find('span', class_='runtime').text.strip(' min')
    genre=movie.find('span', class_='genre').text.strip()
    imdb=movie.find('strong').text
    metascore=movie.find('span', class_='metascore').text.strip()
    description=movie.find('p').find_next_sibling('p').text
    description=description.replace('\n','')
    director_star_element=movie.find('p', class_="").text
    split=director_star_element.split('|')
    director_part = split[0]
    stars_part = split[1]
    director = director_part.replace("Director:", "")
    director = director.replace("Directors:", "")
    director = director.replace("\n", "")
    stars = stars_part.replace("\n", "")
    stars = stars.replace(" Stars:", "").split(", ")
    votes_element = movie.find("span", attrs={"name": "nv", "data-value": True})
    votes = votes_element["data-value"] if votes_element else None
    gross_element = votes_element.find_next_sibling("span", attrs={"name": "nv", "data-value": True})
    gross = gross_element["data-value"] if gross_element else None

    movie_data = {
        "Title": title,
        "Year": year,
        "Runtime (Minutes)":runtime,
        "Genre": genre,
        "IMDB": imdb,
        "Metascore": metascore,
        "Description":description,
        "Director": director.strip(),
        "Stars": ','.join(stars).strip(),
        "Votes": votes,
        "Gross ($)": gross
    }
    movies.append(movie_data)
    

In [ ]:
x=51
for i in range(0,246):
    url='https://www.imdb.com/search/title/?title_type=feature&num_votes=5000,&languages=en&sort=boxoffice_gross_us,desc&start='+str(x)+'&explore=genres&ref_=adv_nxt'
    response= requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    # Find the table element you want to scrape
    movies_divs = soup.find_all("div", class_="lister-item-content")

    for movie in movies_divs:
        title=movie.find('a').text
        year=movie.find('span', class_='lister-item-year').text.strip('()')
        runtime=movie.find('span', class_='runtime').text.strip(' min')
        genre=movie.find('span', class_='genre').text.strip()
        imdb=movie.find('strong').text
        metascore_element=movie.find('span', class_='metascore')
        metascore=metascore_element.text.strip() if metascore_element else None
        description=movie.find('p').find_next_sibling('p').text
        description=description.replace('\n','')
        director_star_element=movie.find('p', class_="").text
        split=director_star_element.split('|')
        director_part = split[0]
        stars_part = split[1]
        director = director_part.replace("Director:", "")
        director = director.replace("Directors:", "")
        director = director.replace("\n", "")
        stars = stars_part.replace("\n", "")
        stars = stars.replace(" Stars:", "").split(", ")
        votes_element = movie.find("span", attrs={"name": "nv", "data-value": True})
        votes = votes_element["data-value"] if votes_element else None
        gross_element = votes_element.find_next_sibling("span", attrs={"name": "nv", "data-value": True})
        gross = gross_element["data-value"] if gross_element else None

        movie_data = {
            "Title": title,
            "Year": year,
            "Runtime (Minutes)":runtime,
            "Genre": genre,
            "IMDB": imdb,
            "Metascore": metascore,
            "Description":description,
            "Director": director.strip(),
            "Stars": ','.join(stars).strip(),
            "Votes": votes,
            "Gross ($)": gross
        }
        movies.append(movie_data)
    x=x+50
data=pd.DataFrame(movies)

In [ ]:
data.head()

In [ ]:
data.describe()

In [ ]:
data.isnull().sum()

In [ ]:
data.dropna(axis=0, inplace=True)

In [ ]:
data.isnull().sum()

In [ ]:
data.duplicated().sum()

In [ ]:
data.drop_duplicates(inplace=True)

In [ ]:
data.duplicated().sum()

In [ ]:
data.describe()

In [ ]:
data.dtypes

In [ ]:
data['Year']=data['Year'].str.replace('\D','').astype(int)

In [ ]:
data[['Runtime (Minutes)','Votes']]=data[['Runtime (Minutes)','Votes']].astype(int)
data[['IMDB','Metascore']]=data[['IMDB','Metascore']].astype(float)

In [ ]:
data['Gross ($)']=data['Gross ($)'].str.replace(',','').astype('int')

In [ ]:
data.dtypes

In [ ]:
data.to_csv('/kaggle/working/movie_data.csv', index=False)